In [9]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv, dotenv_values
import mysql.connector

In [10]:
load_dotenv()
try:
    conn = mysql.connector.connect(
        user = os.getenv("DB_USER"),
        host = os.getenv("DB_HOST"),
        password = os.getenv("DB_PASSWORD"),
        database = os.getenv("DATABASE")
    )
    if conn.is_connected():
        print("Connected to MySQL Database")
except Error as e:
    print("Error while connecting:", e)

Connected to MySQL Database


In [11]:
candidates = pd.read_csv("data/candidate_master.txt", sep="|", header=None, names=["cand_id", "cand_name", "cand_pty_affiliation", "cand_election_yr", "cand_office_st", "cand_office", "cand_office_district", "cand_ici", "cand_status", "cand_pcc", "cand_st1", "cand_st2", "cand_city", "cand_st", "cand_zip"])
candidate_db = candidates[["cand_id", "cand_name", "cand_pty_affiliation", "cand_election_yr", "cand_office_st", "cand_office", "cand_office_district", "cand_ici", "cand_status", "cand_pcc"]]
candidate_db = candidate_db.replace({np.nan: None})
cand_data = [tuple(x) for x in candidate_db.to_numpy()]

In [12]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS Candidates")
cursor.execute("""
               CREATE TABLE Candidates (
                   CandID VARCHAR(50) PRIMARY KEY,
                   CandName VARCHAR(50),
                   CandParty VARCHAR(50),
                   CandElectYr INT,
                   CandStateAbbr VARCHAR(2),
                   CandOffice VARCHAR(1),
                   CandOfficeDist FLOAT,
                   CandIncumbent VARCHAR(1),
                   CandStatus VARCHAR(1),
                   CandPCC VARCHAR(50)
               );
                """)

query = """
        INSERT INTO Candidates(CandID, CandName, CandParty, CandElectYr, CandStateAbbr, 
                                CandOffice, CandOfficeDist, CandIncumbent, CandStatus, CandPCC)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
cursor.executemany(query, cand_data)
conn.commit()

#for _, row in candidate_db.iterrows():
#    print("Inserting:", tuple(row)) 
#    cursor.execute(query, tuple(row))
#    conn.commit()

In [13]:
cursor.execute("SELECT * FROM Candidates LIMIT 10;")
results = cursor.fetchall()
for result in results:
    print(result)
cursor.close()    

('H0AL01055', 'CARL, JERRY LEE, JR', 'REP', 2026, 'AL', 'H', 1.0, 'C', 'C', 'C00697789')
('H0AL01097', 'AVERHART, JAMES', 'DEM', 2024, 'AL', 'H', 2.0, 'C', 'P', 'C00708867')
('H0AL02087', 'ROBY, MARTHA', 'REP', 2020, 'AL', 'H', 2.0, 'I', 'P', 'C00462143')
('H0AL02202', 'HARVEY-HALL, PHYLLIS', 'DEM', 2024, 'AL', 'H', 2.0, 'C', 'P', 'C00728873')
('H0AL05049', 'CRAMER, ROBERT E "BUD" JR', 'DEM', 2008, 'AL', 'H', 5.0, 'I', 'P', 'C00239038')
('H0AL07086', 'SEWELL, TERRI A.', 'DEM', 2026, 'AL', 'H', 7.0, 'I', 'C', 'C00458976')
('H0AR01083', 'CRAWFORD, ERIC ALAN RICK', 'REP', 2026, 'AR', 'H', 1.0, 'I', 'C', 'C00462374')
('H0AR01125', 'SMITH, PRINCELLA D', 'REP', 2010, 'AR', 'H', 1.0, 'O', 'P', 'C00480905')
('H0AR03030', 'WHITAKER, DAVID JEFFREY', 'DEM', 2010, 'AR', 'H', 3.0, 'O', 'P', 'C00468033')
('H0AR03055', 'WOMACK, STEPHEN A THE HON', 'REP', 2026, 'AR', 'H', 3.0, 'I', 'C', 'C00477745')


True